In [1]:
%pylab inline
%config IPCompleter.greedy=True
import numpy as np
from numpy.random import dirichlet, multinomial
import sys
np.random.seed(0)
np.seterr(all='warn')

Populating the interactive namespace from numpy and matplotlib


{'divide': 'warn', 'invalid': 'warn', 'over': 'warn', 'under': 'ignore'}

In [2]:
def visualizeTopics(model, id2word, n_top_words=12):
    for i, topic in enumerate(model.components_):
        print "Topic {}:".format(i+1)
        print " ".join([id2word[j] 
                        for j in topic.argsort()[:-n_top_words - 1:-1]])

In [3]:
""" Generate toy documents """
K = 3
D = 1000
N = 10
V = 12

# 1) Set alpha and beta for generative model
alpha = np.array([2, 2, 2])
beta = np.array([[0.25, 0.25, 0.25, 0.25, 0, 0, 0, 0, 0, 0, 0, 0],
                 [0, 0, 0, 0, 0.25, 0.25, 0.25, 0.25, 0, 0, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0 ,0, 0.25, 0.25, 0.25, 0.25]])

# 2) Using alpha, sample theta_d for each of the D documents
theta = dirichlet(alpha, D)

# 3) Sample topic z_dn for each of the N words of each of the D documents
# 4) Sample word w_dn from topic beta_k corresponding to z_dn
W = []
for d in range(D):
    # Counts of words for each topic in doc d (e.g [4, 3, 3] for N = 10)
    z_d = multinomial(N, theta[d])
    # One hot matrix of words for doc d
    w_d = []
    for k in xrange(K):
        for i in xrange(z_d[k]):
            w_d.append(multinomial(1, beta[k]))
    W.append(np.array(w_d))

In [4]:
from scipy.special import gamma, digamma
from scipy.stats import multinomial as multinomial_distribution
from scipy.stats import dirichlet as dirichlet_distribution
from sklearn.preprocessing import normalize

class LDA():
    def __init__(self, K=3):
        self.K = K
        # Initialize alpha
        self.alpha = np.ones(self.K) / self.K
        
    # W is a list of D numpy arrays of shape (N_d, V)
    def fit(self, W, max_iter=51, threshold=1e-4, verbose=True):
        self.D = len(W)
        self.V = W[0].shape[1]
        
        # Randomly initialize variational parameters
        # TODO best way to initialize gamma, phi ?
        self.gamma = np.random.rand(self.D, self.K)
        self.phi = [np.random.rand(w_d.shape[0], self.K) for w_d in W]
        self.phi = [phi_d / np.sum(phi_d, axis=1, keepdims=True) \
            for phi_d in self.phi]
        
        # Initialize beta
        self._updateBeta(W)
        
        # Keep track of previous value of lower bound on log likelihood
        old_bound = self._computeBound(W)
        
        # Update phi, gamma, beta iteratively while the bound improves
        for i in xrange(max_iter):
            self._updatePhi(W)
            self._updateGamma()
            self._updateBeta(W)
            current_bound = self._computeBound(W)
            if verbose and i % 10 == 0:
                print "Iter {}, bound = {}".format(i, old_bound)
            if (current_bound - old_bound) < threshold:
                break
            old_bound = current_bound
                
        # Return final state of beta and variational parameters
        self.components_ = self.beta # To match sklearn API
        return self.beta, self.gamma, self.phi
        
    def _updateGamma(self):
        for d in xrange(self.D):
            self.gamma[d] = self.alpha + np.sum(self.phi[d], axis=0)
    
    def _updatePhi(self, W):
        for d in xrange(self.D):
            self.phi[d] = W[d].dot(self.beta.T) * \
                np.exp(digamma(self.gamma[d]))
            self.phi[d] = normalize(self.phi[d], norm="l1", axis=1)
    
    def _updateBeta(self, W):
        self.beta = np.concatenate(self.phi, axis=0).T.dot(
            np.concatenate(W, axis=0))
        self.beta = normalize(self.beta, norm="l1", axis=1)
        
    def _computeBound(self, W):
        # Avoid recomputing expensive expressions
        C1 = digamma(self.gamma)
        C2 = digamma(np.sum(self.gamma, axis=1))
        
        # First three terms correspond to expectation of complete
        # log likelihood under variational distribution
        term1 = self.D * (np.log(gamma(np.sum(self.alpha))) - \
            np.sum(np.log(gamma(self.alpha)))) + \
            np.sum(C1 * (self.alpha - 1)) - np.sum(C2) * \
            np.sum(self.alpha - 1)
        term2 = np.sum(np.array([np.sum(phi_d, axis=0) \
            for phi_d in self.phi]) * C1) - \
            np.array([np.sum(phi_d) for phi_d in self.phi]).dot(C2)
        term3 = np.sum(np.log(self.beta) * np.concatenate(
            self.phi, axis=0).T.dot(np.concatenate(W, axis=0)))
        
        # Sum of entropies of dirichlet variational posteriors
        #term4 = -np.sum(np.log(gamma(np.sum(self.gamma, axis=1)))) + \
        #    np.sum(np.log(gamma(self.gamma))) - \
        #    np.sum((self.gamma - 1) * C1) + \
        #    C2.dot(np.sum(self.gamma - 1, axis=1))
        term4 = 0
        for d in xrange(self.D):
            term4 += dirichlet_distribution.entropy(self.gamma[d])
        
        # Sum of entropies of multinomial variational posteriors
        #term5 = -np.sum(np.concatenate(self.phi, axis=0) * \
        #    np.log(np.concatenate(self.phi, axis=0)))
        term5 = np.sum(multinomial_distribution.entropy(
            1, np.concatenate(self.phi, axis=0)))
        
        # Return lower bound on likelihood 
        # (averaged over total number of words)
        L = term1 + term2 + term3 + term4 + term5
        return L / np.concatenate(W, axis=0).shape[0]
        

In [5]:
class LDA_gibbs:

    def __init__(self, n_topics, alpha=None, id2word=False):
        """
        Parameters
        ----------
        n_topics : int
            number of topics for the model
        alpha : array-like, shape=(n_topics)
            hyperparameter for the model
            controls the likelihood of a topic being assigned
            to a document
        id2word : dictionary
            used to map id of word to real world
            helps to monitor progress of learning
        """
        
        self.n_topics = n_topics
        #initialize alpha
        if alpha is None:
            # default value (from Heinrich, 2005)
            self.alpha = np.ones(self.n_topics) / self.n_topics
        else:
            self.alpha = alpha
        self.sumalpha = np.sum(self.alpha)
        self.id2word = id2word
        
    def fit(self, docs, eta=None, niter=100):
        """
        Learning the parameters for the model
        
        Parameters
        ----------
        docs : list of array-like elements, length = n_docs
            each element is an array of dimensions n_words
            by number of words in vocabulary (n_vocab). The 
            number of words varies from one text to the other.
        threshold : float
            used to stop fitting when the model has converged
            to a final state.
        eta : array-like, shape=(n_vocab)
            hyperparameter for the model
            controls the likelihood of a word being assigned 
            to a topic
        niter : int
            number of gibbs sampling iterations for the training
            of the model
        """
        
        self.n_docs = len(docs)
        self.n_vocab = docs[0].shape[1]
        #initialize eta
        if eta is None:
            #default value (from Heinrich, 2005)
            self.eta = np.ones(self.n_vocab) * 0.01
        else:
            self.eta = eta
        self.sumeta = np.sum(self.eta)
        
        #initializing counters
        self._initialize_counters_train(docs)
        
        for i in xrange(niter):
            
            if i%10 == 0:
                #monitoring progress by showing top words for each topic
                print i
                self._estimate_parameters_train()
                if self.id2word:
                    visualizeTopics(self, id2word)

            self._gibbs_sampling_fit_train(docs)

        #update parameters at the end of training
        self._estimate_parameters_train()
        
    def query_sampling(self, queries, niter=100):
        
        self.n_queries = len(queries)
        
        #initializing counters
        self._initialize_counters_train(queries)
        
        for i in xrange(niter):
            self._gibbs_sampling_fit_test(queries)
        
        #update parameters at the end
    
    def perplexity(self, queries):
        
        log_likelihood = 0
        tot_num_words = 0
        for i in range(self.n_queries):
            log_likelihood += np.sum(self.topic_doc_count_test + \
                                     np.log(np.sum(self.beta * self.theta[i])))
            
            tot_num_words += len(queries[i])
        
        perplexity = np.exp(-log_likelihood / tot_num_words)

    
    def _initialize_counters_train(self, docs):
        """Initializing counters randomly for the training phase"""

        #initialize arrays to 0
        self.topic_doc_count = np.zeros((self.n_docs, self.n_topics))
        self.term_topic_count = np.zeros((self.n_topics, self.n_vocab))

        #keep track of topic assignment for each word in memory
        self.topic_assign = []

        for i in xrange(self.n_docs):

            n_words = len(docs[i])

            #sample topic randomly for each word in document
            topic = np.random.randint(self.n_topics, size=n_words)
            self.topic_assign.append(topic)

            #count number of words belonging to each topic
            self.topic_doc_count[i] = np.bincount(topic, minlength=self.n_topics)

            #count word occurence for each topic
            self.term_topic_count += np.array([np.sum(docs[i][np.where(topic==j)], axis=0)\
                                               for j in range(self.n_topics)])

            #sum of number of words in each topic
            self.term_topic_sum = np.sum(self.term_topic_count, axis=1)
            
    def _initialize_counters_test(self, queries):
        """Initializing counters randomly for the test phase"""
        
        #initialize arrays to 0
        self.topic_doc_count_test = np.zeros((self.n_queries,self.n_topics))
        self.term_topic_count_test = np.zeros(self.n_topics, self.n_vocab)
        
        #keep track of topic assignment for each word in memory
        self.topic_assign_test = []
        
        for i in xrange(self.n_queries):
            
            n_words = len(queries[i])
            
            #sample topic randomly for each word in query
            topic = np.random.randint(self.n_topics, size=n_words)
            self.topic_assign_test.append(topic)
            
            #count number fo words belonging to each topic
            self.topic_doc_count_test[i] = np.bincount(topic, minlength=self.n_topics)
            
            #count word occurence for each topic
            self.term_topic_count_test += np.array([np.sum(queries[i][np.where(topic==j)], axis=0)\
                                                    for j in range(self.n_topics)])
            
            #sum of number of words in each topic
            self.term_topic_sum_test = np.sum(self.term_topic_count_test, axis=1)
    
    def _gibbs_sampling_fit_train(self, docs):
        """Update counters for each word using gibbs sampling"""
        
        for i in xrange(self.n_docs):
            
            n_words = len(docs[i])
            
            for j in xrange(n_words):
                
                #decrement associated counters
                self.topic_doc_count[i][self.topic_assign[i][j]] -= 1
                self.term_topic_count[self.topic_assign[i][j]] -= docs[i][j]
                self.term_topic_sum[self.topic_assign[i][j]] -= 1
                
                #sample new topic
                word_idx = np.argmax(docs[i][j])               
                prob = (self.topic_doc_count[i] + self.alpha) * \
                        (self.term_topic_count[:, word_idx] + self.eta[word_idx]) /\
                        (np.sum(self.term_topic_count, axis=1) + self.sumeta) /\
                        ((n_words-1) + self.sumalpha)
                prob /= np.sum(prob)                      #normalize
                prob = np.cumsum(prob)                    #cdf
                self.topic_assign[i][j] = np.argmax(prob > np.random.random())

                #increment associated counters
                self.topic_doc_count[i][self.topic_assign[i][j]] += 1
                self.term_topic_count[self.topic_assign[i][j]] += docs[i][j]  
                self.term_topic_sum[self.topic_assign[i][j]] += 1
                
    def _gibbs_sampling_fit_test(self, queries):
        
        for i in xrange(self.n_queries):
            
            n_words = len(queries[i])
            
            for j in xrange(n_words):
                
                #decrement associated counters
                self.topic_doc_count_test[i][self.topic_assign[i][j]] -= 1
                self.term_topic_count_test[self.topic_assign_test] -= queries[i][j]
                self.term_topic_sum_test[self.topic_assign_test[i][j]] -= 1
                
                #sample new topic
                word_idx = np.argmax(queries[i][j])
                
                prob = (self.topic_doc_count_test[i] + self.alpha) * \
                        (self.term_topic_count_test[:,word_idx] + \
                             self.term_topic_count[:,word_idx] + \
                             self.eta[word_idx]) /\
                        (np.sum(self.term_topic_count_test, axis=1) + \
                             self.term_topic_sum + \
                             self.sumeta) /\
                        ((n_words + self.sumalpha))
                prob /= np.sum(prob)              #normalize
                prob = np.cumsum(prob)            #cdf
                self.topic_assign_test[i][j] = np.argmax(prob > np.random.random())
                
                #increment associated counters
                self.topic_doc_count_test[i][self.topic_assign_test[i][j]] += 1
                self.term_topic_count_test[self.topic_assign_test[i][j]] += queries[i][j]
                self.term_topic_sum_test[self.topic_assign[i][j]] += 1
                
    def _estimate_parameters_train(self):
        """Estimate parameters used to make predictions"""
        
        self.beta = np.divide((self.term_topic_count + self.eta).T, 
                             np.sum((self.term_topic_count + self.eta), axis=1)).T
        
        self.theta = np.divide((self.topic_doc_count + self.alpha).T, 
                             np.sum((self.topic_doc_count + self.alpha), axis=1)).T
        
        self.components_ = self.beta
    
    def _estimate_parameters_test(self):
        
                                         
        self.theta_test = np.divide((self.topic_doc_count_test + self.alpha).T,
                                   np.sum((self.topic_doc_count_test + self.alpha), axis=1)).T
        

In [6]:
""" Simple test on toy documents """
model = LDA()
beta, _, _ = model.fit(W)

# We more or less retrieve the beta used in the generating process
print beta

'\nmodel = LDA()\nbeta, _, _ = model.fit(W)\n\n# We more or less retrieve the beta used in the generating process\nprint beta\n'

In [7]:
""" Get the 20 news groups data """
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
import pickle

cats = ['alt.atheism', 'sci.space']
newsgroups = fetch_20newsgroups(shuffle=True, random_state=1, subset="train",
                                remove=("headers", "footers", "quotes"))#, categories=cats)

In [8]:
""" Prepare input for sklearn (counts) """
n_features = 2000
vectorizer = CountVectorizer(max_features=n_features, stop_words="english")

# Word counts per document matrix (input for sklearn)
W_counts = vectorizer.fit_transform(newsgroups.data)

# Keep track of vocabulary to visualize top words of each topic
vocabulary = vectorizer.get_feature_names()

In [9]:
""" Prepare input for our model (one hot vectors) """
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from gensim.corpora import Dictionary
import string

# 1) Tokenize document
# 2) Remove stop words
# 3) Lemmatize
lemmatizer = WordNetLemmatizer()
#stemmer = PorterStemmer()
# TODO : treat special list better
# TODO See what count vectorizer does
special = ["''", "'s", "``", "n't", "...", "--"]
stop = set(stopwords.words('english') + \
           list(string.punctuation) + special)
def prepare_document(doc):
    words = [lemmatizer.lemmatize(w) for w in word_tokenize(doc.lower()) 
             if w not in stop] 
    return words


# List of documents (each document is a list of word tokens)
texts = [prepare_document(text) for text in newsgroups.data]

# Create a gensim dictionary for our corpus
dic = Dictionary(texts)

# Keep only k most frequent words
n_features = 2000
dic.filter_extremes(keep_n=n_features)
vocab_size = len(dic.token2id) # Vocabulary size

# List of documents (each document is now a list of word indexes)
# We have removed all words not in the k most frequent words
texts_idx = [[dic.token2id[word] for word in text 
              if word in dic.token2id] for text in texts]

# Convert each index to a one hot vector
W = np.array([np.eye(vocab_size)[text] for text in texts_idx if len(text) > 0])

# Keep track of id to word mapping to visualize top words of topics
id2word = dict([[v, k] for k, v in dic.token2id.items()])

pickle.dump(W, open("W.p", "wb"))
pickle.dump(id2word, open("mapping.p", "wb"))

'\nfrom nltk.tokenize import word_tokenize\nfrom nltk.corpus import stopwords\nfrom nltk.stem import WordNetLemmatizer\nfrom nltk.stem.porter import PorterStemmer\nfrom gensim.corpora import Dictionary\nimport string\n\n# 1) Tokenize document\n# 2) Remove stop words\n# 3) Lemmatize\nlemmatizer = WordNetLemmatizer()\n#stemmer = PorterStemmer()\n# TODO : treat special list better\n# TODO See what count vectorizer does\nspecial = ["\'\'", "\'s", "``", "n\'t", "...", "--"]\nstop = set(stopwords.words(\'english\') +            list(string.punctuation) + special)\ndef prepare_document(doc):\n    words = [lemmatizer.lemmatize(w) for w in word_tokenize(doc.lower()) \n             if w not in stop] \n    return words\n\n\n# List of documents (each document is a list of word tokens)\ntexts = [prepare_document(text) for text in newsgroups.data]\n\n# Create a gensim dictionary for our corpus\ndic = Dictionary(texts)\n\n# Keep only k most frequent words\nn_features = 2000\ndic.filter_extremes(keep_

In [10]:
from sklearn.decomposition import LatentDirichletAllocation

n_topics = 20

def visualizeTopics(model, id2word, n_top_words=12):
    for i, topic in enumerate(model.components_):
        print "Topic {}:".format(i+1)
        print " ".join([id2word[j] 
                        for j in topic.argsort()[:-n_top_words - 1:-1]])

# Compare with sklearn implementation
print "Sklearn model"
lda_sklearn = LatentDirichletAllocation(n_topics=n_topics, 
                                        learning_method="batch")
lda_sklearn.fit(W_counts)
visualizeTopics(lda_sklearn, vocabulary)

# Our model
print "\nOur model"
lda = LDA(K=n_topics)
W = pickle.load(open("W.p", "rb"))
id2word = pickle.load(open("mapping.p", "rb"))
lda.fit(W, verbose=True)
visualizeTopics(lda, id2word)

Sklearn model


/usr/local/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic 1:
years time people like work ago new think problem just know long
Topic 2:
edu com cs uk ca apr ac david article university pain soon
Topic 3:
drive scsi card disk hard drives memory bus controller ram use board
Topic 4:
software image bit color use data used sale video graphics jpeg monitor
Topic 5:
car good like just use power used new bike don ve buy
Topic 6:
cx ah w7 chz lk hz c_ mv uw t7 34u ck
Topic 7:
windows window problem thanks using use does help know file dos set
Topic 8:
ax max g9v b8f a86 pl 145 1d9 0t 1t giz bhj
Topic 9:
available edu ftp version pub server motif graphics widget mit com sun
Topic 10:
db armenian turkish israel armenians people war jews israeli turkey government greek
Topic 11:
god jesus people does believe bible christian say true church think question
Topic 12:
don just like think know people good really say want make does
Topic 13:
said didn people know did went just time don came saw told
Topic 14:
government key gun law mr people president en

'\nprint "\nOur model"\nlda = LDA(K=n_topics)\nW = pickle.load(open("W.p", "rb"))\nid2word = pickle.load(open("mapping.p", "rb"))\nlda.fit(W, verbose=True)\nvisualizeTopics(lda, id2word)\n'

In [ ]:
#results for the LDA implementation using Gibbs sampling
n_topics = 2
lda_gibbs = LDA_gibbs(n_topics)
lda_gibbs.fit(W)
        
visualizeTopics(lda_gibbs, id2word)

In [ ]:
#test models with test set
newsgroups_test = fetch_20newsgroups(shuffle=True, random_state=1, subset="test",
                                remove=("headers", "footers", "quotes"), categories=cats)

In [ ]:
""" Visualize most likely topic assignment for each word 
    [if p(most likely topic) < threshold, don't assign a topic]
    and visualize expected distribution over topics
    Inputs:
        model, our trained lda model
        document_idx, index of document to visualize
        id2word, mapping from word index to word string
        topic2word, mapping from topic index to topic string
        threshold, float in [0.0, 1.0]
    """
def visualizeWordAssignments(model, document_idx, id2word, 
                             topic2word, threshold=0.9):
    # We add a column with value threshold so that if the index of this 
    # column gets selected by the argmax operation, we don't assign a topic
    col = np.ones((model.phi[document_idx].shape[0], 1)) * threshold
    col_index = model.phi[document_idx].shape[1]
    topic2word[col_index] = " "
    word_topic_probas = np.concatenate(
        (model.phi[document_idx], col), axis=1)
    
    # Expected distribution over topics
    doc_topic_probas = dirichlet_distribution.mean(
        model.gamma[document_idx])
    
    print "DOCUMENT:"
    print newsgroups.data[document_idx]
    
    print "\nEXPECTED PROBA DISTRIB OVER TOPICS:"
    for i in xrange(len(doc_topic_probas)):
        print "{}: {}".format(topic2word[i], doc_topic_probas[i])
        
    print "\nWORD TOPIC ASSIGNMENTS:"
    for word_idx, topic_idx in zip(texts_idx[document_idx], 
                                   np.argmax(word_topic_probas, axis=1)): 
        print "{} -> {}".format(id2word[word_idx], topic2word[topic_idx])

In [ ]:
topic2word = {0: "topic 1", 1: "topic 2", 2: "topic 3", 3: "topic 4"}
visualizeWordAssignments(lda, 0, id2word, topic2word)

In [ ]:
topic2word = {0: "space", 1: "religion"}
visualizeWordAssignments(lda, 6, id2word, topic2word)

In [ ]:
# TODO Perform inference on new documents
# only E-step (estimate phi and gamma) until the end